In [20]:
# pip install jupyter openai minsearch requests

In [21]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [22]:
from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [23]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [24]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [25]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [26]:

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [29]:
rag("How do I patch KDE under FreeBSD?")

"I'm sorry, but there is no information provided in the context regarding how to patch KDE under FreeBSD. Please refer to relevant documentation or resources that might provide guidance on this topic."

In [30]:
print(llm("How do I patch KDE under FreeBSD?"))

Patching KDE under FreeBSD typically involves a few steps, which include updating your ports tree and applying any necessary patches to the KDE components. Below are general steps you can follow to apply patches to KDE:

### Step 1: Update Ports Tree

1. **Install Ports Collection (if you haven't already):**
   If you do not have the ports collection set up, you can do so with:
   ```bash
   portsnap fetch extract
   ```

2. **Update the Ports Collection:**
   To update your ports tree, use:
   ```bash
   portsnap fetch update
   ```

### Step 2: Locate the KDE Port

Before you apply patches, identify the KDE port you are interested in. KDE components are often located in the `/usr/ports/x11/kde*` directory, where `kde*` corresponds to various modules (e.g., `kde5`, `kde5-applications`, etc.).

### Step 3: Navigate to the KDE Port Directory

Change directory to the specific KDE port you want to patch. For example:

```bash
cd /usr/ports/x11/kde5
```

### Step 4: Apply the Patch

1. **L